In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (10, 10)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset = pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv')

In [ ]:
dataset.shape

In [ ]:
dataset.describe()

In [ ]:
dataset.isnull().sum() / len(dataset) * 100

In [ ]:
dataset = dataset.drop(['Date', 'Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis = 1)

In [ ]:
dataset.shape

In [ ]:
dataset['RainTomorrow'].value_counts() / len(dataset)

In [ ]:
dataset_cat = dataset.select_dtypes('O')
dataset_cat

In [ ]:
dataset_num = dataset.select_dtypes(float)
dataset_num

In [ ]:
print(dataset_cat.isnull().sum())
print(dataset_num.isnull().sum())

In [ ]:
num_cols = dataset_num.columns
cat_cols = dataset_cat.columns
num_cols, cat_cols

In [ ]:
dataset_num.hist(bins = 100)

In [ ]:
dataset['Rainfall'].std()

In [ ]:
dataset['Rainfall'].hist(bins = 100)

In [ ]:
for i in range(dataset_num.shape[1]):
    plt.figure(figsize = (6, 6))
    sns.boxplot(dataset_num.iloc[:, i])
    plt.show()

In [ ]:
dataset_cat = dataset.select_dtypes('O')
dataset_cat

In [ ]:
dataset_cat['Location'].value_counts()

In [ ]:
dataset['WindGustDir'].value_counts()

In [ ]:
dataset_cat['RainToday'].value_counts()

In [ ]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train = pd.DataFrame(X_train, columns = dataset.columns[:-1])
X_train

In [ ]:
X_test = pd.DataFrame(X_test, columns = dataset.columns[:-1])
X_test

In [ ]:
X_train_num = X_train.select_dtypes(float)
X_train_num

In [ ]:
X_train_cat = X_train.select_dtypes('O')
X_train_cat

In [ ]:
X_train_num.shape

In [ ]:
from sklearn.impute import SimpleImputer
sim_num = SimpleImputer(strategy = 'median')
X_train_num = sim_num.fit_transform(X_train_num)
X_train_num = pd.DataFrame(X_train_num, columns = num_cols)
X_train_num

In [ ]:
sim_num.statistics_

In [ ]:
X_train_num.isnull().sum()

In [ ]:
cat_cols[:-1]

In [ ]:
X_train_cat

In [ ]:
from sklearn.impute import SimpleImputer
sim_cat = SimpleImputer(strategy = 'most_frequent')
X_train_cat = sim_cat.fit_transform(X_train_cat)
X_train_cat = pd.DataFrame(X_train_cat, columns = cat_cols[:-1])
X_train_cat

In [ ]:
X_train_cat.isnull().sum()

In [ ]:
sim_cat.statistics_

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_num = sc.fit_transform(X_train_num)
X_train_num = pd.DataFrame(X_train_num, columns = num_cols)

In [ ]:
X_train_num

In [ ]:
X_train_cat = pd.get_dummies(X_train_cat)
X_train_cat

In [ ]:
X_train_final = pd.concat([X_train_num, X_train_cat], axis = 1)
X_train_final

In [ ]:
X_train_f = X_train_final.values
X_train_f

In [ ]:
y_train

In [ ]:
y_train.isnull().sum()

In [ ]:
y_train.shape

In [ ]:
sim_mode = SimpleImputer(strategy = 'most_frequent')
y_train = sim_mode.fit_transform(y_train.values.reshape(-1, 1))
y_train

In [ ]:
y_train.shape

In [ ]:
sim_mode.statistics_

In [ ]:
from sklearn.preprocessing import LabelEncoder
lab = LabelEncoder()
y_train = lab.fit_transform(y_train)

In [ ]:
y_train.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtf = DecisionTreeClassifier()

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5)

In [ ]:
scores_dt = []

In [ ]:
X_train_f.shape

In [ ]:
y_train.shape

In [ ]:
for train_idx, test_idx in kf.split(X_train_f):
    X_train_idx, X_test_idx = X_train_f[train_idx], X_train_f[test_idx]
    y_train_idx, y_test_idx = y_train[train_idx], y_train[test_idx]
    dtf.fit(X_train_idx, y_train_idx)
    scores_dt.append(dtf.score(X_test_idx, y_test_idx))

In [ ]:
scores_dt

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

log_reg = LogisticRegression()
nb = GaussianNB()
rf = RandomForestClassifier()
ada = AdaBoostClassifier()
gb = GradientBoostingClassifier()

In [ ]:
scores_lr = []

for train_idx, test_idx in kf.split(X_train_f):
    X_train_idx, X_test_idx = X_train_f[train_idx], X_train_f[test_idx]
    y_train_idx, y_test_idx = y_train[train_idx], y_train[test_idx]
    log_reg.fit(X_train_idx, y_train_idx)
    scores_lr.append(log_reg.score(X_test_idx, y_test_idx))

In [ ]:
scores_nb = []

for train_idx, test_idx in kf.split(X_train_f):
    X_train_idx, X_test_idx = X_train_f[train_idx], X_train_f[test_idx]
    y_train_idx, y_test_idx = y_train[train_idx], y_train[test_idx]
    nb.fit(X_train_idx, y_train_idx)
    scores_nb.append(nb.score(X_test_idx, y_test_idx))

In [ ]:
#scores_dt, 
scores_lr, scores_nb

In [ ]:
scores_rf = []

for train_idx, test_idx in kf.split(X_train_f):
    X_train_idx, X_test_idx = X_train_f[train_idx], X_train_f[test_idx]
    y_train_idx, y_test_idx = y_train[train_idx], y_train[test_idx]
    rf.fit(X_train_idx, y_train_idx)
    scores_rf.append(rf.score(X_test_idx, y_test_idx))

In [ ]:
scores_ada = []

for train_idx, test_idx in kf.split(X_train_f):
    X_train_idx, X_test_idx = X_train_f[train_idx], X_train_f[test_idx]
    y_train_idx, y_test_idx = y_train[train_idx], y_train[test_idx]
    ada.fit(X_train_idx, y_train_idx)
    scores_ada.append(ada.score(X_test_idx, y_test_idx))

In [ ]:
scores_gb = []

for train_idx, test_idx in kf.split(X_train_f):
    X_train_idx, X_test_idx = X_train_f[train_idx], X_train_f[test_idx]
    y_train_idx, y_test_idx = y_train[train_idx], y_train[test_idx]
    gb.fit(X_train_idx, y_train_idx)
    scores_gb.append(gb.score(X_test_idx, y_test_idx))

In [ ]:
scores_rf, scores_ada, scores_gb

In [ ]:
for estimator in [log_reg, dtf, nb]:
    print(estimator)
    for train_idx, test_idx in kf.split(X_train_f):
        X_train_idx, X_test_idx = X_train_f[train_idx], X_train_f[test_idx]
        y_train_idx, y_test_idx = y_train[train_idx], y_train[test_idx]
        estimator.fit(X_train_idx, y_train_idx)
        print(estimator.score(X_test_idx, y_test_idx))

In [ ]:
y_score_lr = log_reg.decision_function(X_train_f)
y_score_lr

In [ ]:
y_score_rf = rf.predict_proba(X_train_f)
y_score_rf = y_score_rf[:, 1]
y_score_rf

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
print(roc_auc_score(y_train, y_score_lr))
print(roc_auc_score(y_train, y_score_rf))

In [ ]:
fpr_lr, tpr_lr, th_lr = roc_curve(y_train, y_score_lr)
fpr_rf, tpr_rf, th_rf = roc_curve(y_train, y_score_rf)

In [ ]:
plt.plot(fpr_lr, tpr_lr, label = "Logistic Regression")
plt.plot(fpr_rf, tpr_rf, label = "Random Forest")
plt.plot([0, 1], [0, 1], 'k:')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.title('ROC Curve for Rainfall Prediction')
plt.legend()
plt.show()

In [ ]:
X_test

In [ ]:
X_test_num = X_test.select_dtypes(float)
X_test_num

In [ ]:
print(sim_num.statistics_)
print(sim_cat.statistics_)
print(sim_mode.statistics_)

In [ ]:
X_test_num = sim_num.transform(X_test_num)
X_test_num

In [ ]:
X_test_num = sc.transform(X_test_num)
X_test_num

In [ ]:
X_test_num = pd.DataFrame(X_test_num, columns = num_cols)
X_test_num

In [ ]:
X_test_cat = X_test.select_dtypes('O')
X_test_cat

In [ ]:
cat_cols

In [ ]:
X_test_cat = pd.DataFrame(sim_cat.transform(X_test_cat), columns = cat_cols[:-1])
X_test_cat

In [ ]:
X_test_cat = pd.get_dummies(X_test_cat)
X_test_cat

In [ ]:
X_test_final = pd.concat([X_test_num, X_test_cat], axis = 1)
X_test_final

In [ ]:
y_test = sim_mode.transform(y_test.values.reshape(-1, 1))
y_test

In [ ]:
y_test = lab.transform(y_test)
y_test

In [ ]:
Y_pred_lr = log_reg.predict(X_test_final)
Y_pred_rf = rf.predict(X_test_final)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score

In [ ]:
accuracy_score(y_test, Y_pred_lr)

In [ ]:
accuracy_score(y_test, Y_pred_rf)

In [ ]:
print(precision_score(y_test, Y_pred_rf))
print(precision_score(y_test, Y_pred_lr))
print(recall_score(y_test, Y_pred_rf))
print(recall_score(y_test, Y_pred_lr))

In [ ]:
y_score_lr_test = log_reg.decision_function(X_test_final)
y_score_lr_test

In [ ]:
y_score_rf_test = rf.predict_proba(X_test_final)
y_score_rf_test = y_score_rf_test[:, 1]
y_score_rf_test

In [ ]:
print(roc_auc_score(y_test, y_score_lr_test))
print(roc_auc_score(y_test, y_score_rf_test))

In [ ]:
fpr_lr_test, tpr_lr_test, th_lr_test = roc_curve(y_test, y_score_lr_test)
fpr_rf_test, tpr_rf_test, th_rf_test = roc_curve(y_test, y_score_rf_test)

In [ ]:
plt.plot(fpr_lr, tpr_lr, label = "Logistic Regression on the Train Set")
plt.plot(fpr_rf, tpr_rf, label = "Random Forest on the Train Set")
plt.plot(fpr_lr_test, tpr_lr_test, label = "Logistic Regression on the Test Set")
plt.plot(fpr_rf_test, tpr_rf_test, label = "Random Forest on the Test Set")
plt.plot([0, 1], [0, 1], 'k:')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.title('ROC Curve for Rainfall Prediction')
plt.legend()
plt.show()